In [1]:
# CHALLENGE PART 2 Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gmaps
# Immport the requests log
import requests
requests.__version__

#Import the API Key
from config import weather_api_key
from config import g_key

In [2]:
# CHALLENGE PART 2: Import the CSV as a DF
weather_py_df = pd.read_csv("WeatherPy_database.csv")
weather_py_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Hobart,AU,-42.88,147.33,46.00,87,75,5.82,broken clouds,0,0
1,1,Cap Malheureux,MU,-19.98,57.61,72.00,74,19,8.01,few clouds,0,0
2,2,Esperance,AU,-33.87,121.90,48.20,61,0,20.80,clear sky,0,0
3,3,Busselton,AU,-33.65,115.33,59.00,68,98,12.80,light rain,0,0
4,4,Mataura,NZ,-46.19,168.86,44.11,84,75,4.54,broken clouds,0,0


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
max_temp = weather_py_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [5]:
#CHALLENGE PART 2: Ask the Customer for the minimum and maximum temperature preferences
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [6]:
# CHALLENGE PART 2: Ask the customer if they would prefer for it to be raining or snowing
rain_input = input("Do you want it to be raining? (yes/no) ")
snow_input = input("Do you want it to be snowing? (yes/no) ")

In [7]:
10
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = weather_py_df .loc[(weather_py_df["Max Temp"] <= max_temp) &
                                       (weather_py_df["Max Temp"] >= min_temp) &
                                       (weather_py_df["Rain (inches)"] == 0 if rain_input == "no" else weather_py_df["Rain (inches)"] > 0) &
                                       (weather_py_df["Snow (inches)"] == 0 if snow_input == "no" else weather_py_df["Snow (inches)"] > 0)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
9,9,Makakilo City,US,21.35,-158.09,86.00,51,75,6.93,broken clouds,0,0
16,16,Atuona,PF,-9.80,-139.03,79.52,77,1,19.64,clear sky,0,0
17,17,Miri,MY,4.41,114.01,82.40,83,20,3.36,few clouds,0,0
18,18,Kavieng,PG,-2.57,150.80,84.18,68,98,5.08,overcast clouds,0,0
21,21,Marsh Harbour,BS,26.54,-77.06,82.65,73,85,15.28,overcast clouds,0,0
22,22,Rockport,US,28.02,-97.05,89.60,70,1,9.17,clear sky,0,0
24,24,Padang,ID,-0.95,100.35,77.34,81,100,4.14,light rain,0,0
25,25,Diffa,NE,13.32,12.61,81.18,58,0,9.33,clear sky,0,0
29,29,Basco,PH,20.45,121.97,84.83,81,0,12.59,clear sky,0,0
30,30,Bengkulu,ID,-3.80,102.27,76.26,86,94,2.82,overcast clouds,0,0


In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng",]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Makakilo City,US,86.00,broken clouds,21.35,-158.09,
16,Atuona,PF,79.52,clear sky,-9.80,-139.03,
17,Miri,MY,82.40,few clouds,4.41,114.01,
18,Kavieng,PG,84.18,overcast clouds,-2.57,150.80,
21,Marsh Harbour,BS,82.65,overcast clouds,26.54,-77.06,
22,Rockport,US,89.60,clear sky,28.02,-97.05,
24,Padang,ID,77.34,light rain,-0.95,100.35,
25,Diffa,NE,81.18,clear sky,13.32,12.61,
29,Basco,PH,84.83,clear sky,20.45,121.97,
30,Bengkulu,ID,76.26,overcast clouds,-3.80,102.27,


In [9]:
# CHALLENGE PART 2 parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [10]:
# CHALLENGE PART 2: Get Hotels for the list
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_name = hotels["results"][0]["name"]
        #print(hotels["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except (KeyError,IndexError):
        print("Hotel not found... Skipping")

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [11]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Makakilo City,US,86.00,broken clouds,21.35,-158.09,Marriott's Ko Olina Beach Club
16,Atuona,PF,79.52,clear sky,-9.80,-139.03,Villa Enata
17,Miri,MY,82.40,few clouds,4.41,114.01,"Mega Hotel, Miri"
18,Kavieng,PG,84.18,overcast clouds,-2.57,150.80,Nusa Island Retreat
21,Marsh Harbour,BS,82.65,overcast clouds,26.54,-77.06,Abaco Beach Resort


In [12]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# CHALLENGE PART 2: Save the new DF as a CSV
output_file = "WeatherPy_vacation.csv"
#Export the df to the output
hotel_df.to_csv(output_file, index_label = "City_ID")

In [15]:
 #CHALLENGE PART 2: Add a Marker Layer Map with a Pop-up Marker for each city that includes: Hotel Name, City, Country, Current Weather Description with Max Tempurature
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))